In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
# df=pd.read_csv("./pluto_rodent_cat.csv")
pl=pd.read_csv("./pluto_add_indicater.csv")

In [9]:
from sklearn import svm

In [10]:
X = [['0'], ['1'], ['2'], ['3']]
Y = [0, 1, 2, 3]
clf = svm.NuSVC(decision_function_shape='ovo')
clf.fit(X, Y)
clf.predict([['-1']])

array([0])

# Pick Attributes

In [11]:
pluto_cols_needed = [
                    "bldgclass",
                    "numbldgs",
                    "unitsres",
                    "lottype",
                    "assesstot",
                    "histdist",
                    "landmark",
                    "bbl_indicator"
                    ]
pl = pl[pluto_cols_needed]
# pl = pl.astype('str')

# Categorized

In [15]:
pl['bldgclass'] = pl['bldgclass'].astype(str).str[0]
pl.loc[(pl["bldgclass"]=='n'), "bldgclass"] = 0
def label(df):
    if df['bldgclass'] == 0:
        return 0
    else:
        return ord(df['bldgclass']) - ord('A') + 1
pl['bldgclass']=pl.apply(label,axis=1)

In [16]:
bins = [0,1,2,5,10,100,1850 ]
pl['numbldgs'] = pd.cut(pl['numbldgs'], bins, right = False, include_lowest=True, labels = [0,1,2,3,4,5])

In [25]:
pl.loc[(pl["numbldgs"].isnull()), "numbldgs"] = 0

In [38]:
pl.loc[(pl["unitsres"].isnull()), "unitsres"] = 0

In [39]:
pl.loc[(pl["lottype"].isnull()), "lottype"] = 0

In [17]:
def label(df):
    if df['assesstot'] == 0:
        return 0
    if df['assesstot']<0.5*1e7:
        return 1
    elif df['assesstot']<1e7:
        return 2
    elif df['assesstot']<1.5*1e7:
        return 3
    elif df['assesstot']<2*1e7:
        return 4
    elif df['assesstot']<4*1e7:
        return 5
    elif df['assesstot']<1e8:
        return 6
    elif df['assesstot']<0.5*1e9:
        return 7
    elif df['assesstot']>=0.5*1e9:
        return 8
    else:
        return 0
pl['assesstot']=pl.apply(label,axis=1)

In [29]:
pl.loc[(pl["assesstot"].isnull()), "assesstot"] = 0

In [18]:
pl['histdist'] = pl['histdist'].astype('str')
pl['landmark'] = pl['landmark'].astype('str')
def label(df):
    if df['histdist']=="nan":
        return 0
    else:
        return 1
pl['histdist']=pl.apply(label,axis=1)
def label(df):
    if df['landmark']=="nan":
        return 0
    else:
        return 1
pl['landmark']=pl.apply(label,axis=1)
pl.head()

,bldgclass,numbldgs,unitsres,lottype,histdist,assesstot,landmark,bbl_indicator
0,1,1,1.0,5.0,1,1.0,0,1
1,2,1,2.0,5.0,1,1.0,0,1
2,3,1,9.0,5.0,1,1.0,0,1
3,3,1,10.0,5.0,1,1.0,0,1
4,22,0,0.0,5.0,1,1.0,0,1


In [40]:
pl_ls = pl.values.tolist()
len(pl_ls)
pl_ls[:5]

[[1.0, 1.0, 1.0, 5.0, 1.0, 1.0, 0.0, 1.0],
 [2.0, 1.0, 2.0, 5.0, 1.0, 1.0, 0.0, 1.0],
 [3.0, 1.0, 9.0, 5.0, 1.0, 1.0, 0.0, 1.0],
 [3.0, 1.0, 10.0, 5.0, 1.0, 1.0, 0.0, 1.0],
 [22.0, 0.0, 0.0, 5.0, 1.0, 1.0, 0.0, 1.0]]

In [41]:
pl.value_counts()

bldgclass  numbldgs  unitsres  lottype  histdist  assesstot  landmark  bbl_indicator
1          1         1.0       5.0      1         1.0        0         0                142632
2          1         2.0       5.0      1         1.0        0         0                102212
1          2         1.0       5.0      1         1.0        0         0                 87556
2          1         2.0       5.0      1         1.0        0         1                 63049
           2         2.0       5.0      1         1.0        0         0                 47610
                                                                                         ...  
4          3         252.0     5.0      1         2.0        0         1                     1
                                                                       0                     1
                               1.0      1         2.0        0         1                     1
                     245.0     1.0      1         2.0       

In [43]:
pl["unitsres"].isnull().sum()

0

# Split Dataset

In [9]:
pl['bbl_indicator'].value_counts()

0    545300
1    313768
Name: bbl_indicator, dtype: int64

In [10]:
pl1 = pl[pl['bbl_indicator']==1]
pl0 = pl[pl['bbl_indicator']==0]

In [11]:
pl1['bbl_indicator'].value_counts()

1    313768
Name: bbl_indicator, dtype: int64

In [12]:
pl0['bbl_indicator'].value_counts()

0    545300
Name: bbl_indicator, dtype: int64

In [13]:
split_point = int(pl1.shape[0]*0.6)

In [14]:
# import random

In [15]:
# pred = pl0[split_point:]
# pl0 = pl0[:split_point]
# train = pd.concat([pl0, pl1[:split_point]])
# train_ls = train.values.tolist()
# # random.shuffle(train_ls)
# x_train, y_train = np.split(train_ls, (train.shape[1]-1,), axis=1)

In [16]:
# test = pl1[split_point:]
# test_ls = test.values.tolist()
# # random.shuffle(test_ls)
# x_test, y_test = np.split(test_ls, (test.shape[1]-1,), axis=1)

# Prediction

In [44]:
from sklearn.model_selection import train_test_split
x, y = np.split(pl_ls, (pl.shape[1]-1,), axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [45]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame.from_dict(report)

,0.0,1.0,accuracy,macro avg,weighted avg
precision,0.685348,0.423133,0.563144,0.554240,0.589585
recall,0.576481,0.539961,0.563144,0.558221,0.563144
f1-score,0.626218,0.474461,0.563144,0.550340,0.570795
support,218132.000000,125496.000000,0.563144,343628.000000,343628.000000


In [46]:
cnt = 0
for each in y_test:
    if each == 1:
        cnt += 1
cnt

125496

In [47]:
len(y_test)

343628

In [48]:
cnt = 0
for each in y_pred:
    if each == 1:
        cnt += 1
cnt

160146